In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install faker 

In [ ]:
import faker
import pandas as pd
import random

# Create a Faker instance
fake = faker.Faker()

# Set a seed for reproducibility
random.seed(42)

# Function to generate random data for each scenario
def generate_data(num_records):
    data = []

    for user_id in range(1, num_records + 1):
        # User ID and Name
        name = f"Name {user_id}"

        # Linked Accounts
        linked_accounts = 3 if random.random() < 0.1 else random.randint(1, 2)

        # Money comes from different sources
        num_sources = 3 if random.random() < 0.1 else random.randint(1, 2)
        sources = [fake.credit_card_number(card_type='mastercard') for _ in range(num_sources)]

        # Total credit amount
        total_credit_amount = round(random.uniform(10000, 300000), 2)

        # Only 20% of credit amount above 150k
        if random.random() < 0.1:
            total_credit_amount = max(total_credit_amount, 150000)

        # Money transfer details
        num_transfer_accounts = 3 if random.random() < 0.1 else 1
        transfer_accounts = [fake.credit_card_number(card_type='visa') for _ in range(num_transfer_accounts)]

        # Transaction amount
        transaction_amount = round(total_credit_amount * random.uniform(0.2, 1), 2)

        # Only 20% of transaction amount above 150k
        if random.random() < 0.1:
            transaction_amount = max(transaction_amount, 150000)

        # Additional parameters for Kaggle-style dataset
        employment_status = random.choice(['Employed', 'Unemployed', 'Student'])
        education_level = random.choice(['High School', 'Bachelor', 'Master', 'PhD'])
        marital_status = random.choice(['Single', 'Married', 'Divorced'])

        # Fraud Indicator
        fraud_conditions_met = sum([
            linked_accounts == 3,
            total_credit_amount > 150000 and num_sources == 3,
            transaction_amount > 150000 and num_transfer_accounts == 3,
            total_credit_amount > 150000 and num_sources == 1 and num_transfer_accounts == 3,
            transaction_amount > 150000 and num_transfer_accounts == 1 and num_sources == 3,
            total_credit_amount > 150000 and num_sources == 1 and num_transfer_accounts == 1,
            transaction_amount > 150000 and num_transfer_accounts == 1 and num_sources == 1,
            total_credit_amount > 150000 and num_sources == 2 and num_transfer_accounts == 2,
            transaction_amount > 150000 and num_transfer_accounts == 2 and num_sources == 2,
            employment_status == 'Unemployed',
        ])

        fraud_indicator = 1 if fraud_conditions_met >= 2 else 0
        data.append([user_id, name, linked_accounts, sources,
                     total_credit_amount, transfer_accounts, transaction_amount,
                     employment_status, education_level, marital_status, fraud_indicator])

    return data



In [ ]:
# Generate dataset with 20000 records
num_records = 20000
columns = ["User ID", "Name", "Linked Accounts", "Money Sources",
           "Total Credit Amount", "Transfer Accounts", "Transaction Amount",
           "Employment Status", "Education Level", "Marital Status", "Fraud Indicator"]
dataset = pd.DataFrame(generate_data(num_records), columns=columns)

# Save dataset to CSV file in Kaggle-style directory
file_path = "/kaggle/working/k.csv"
dataset.to_csv(file_path, index=False)

print(f"Dataset saved at: {file_path}")
